# 형태소 분석 및 SentencePiece Unigram

In [ ]:
# --- 설치 ---
!pip -q install konlpy mecab-python3 sentencepiece pandas


[알림] Mecab 사용 불가 → Okt로 진행합니다.
사용 형태소 분석기: okt
1단계(형태소): ['지피다', '신조어', '자다', '처리', '하다', '확인', '하다', '보다']
2단계(형태소→SP): ['▁', '지', '피', '다', '▁', '신', '조', '어', '▁', '자', '다', '▁', '처', '리', '▁', '하', '다', '▁', '확', '인', '▁', '하', '다', '▁', '보', '다']  | 길이: 26


,text,len_char,len_morph,len_morph_sp
0,"오늘은 비가 오지만, 축구 경기는 예정대로 진행됐다.",23,8,22
1,한글 토큰화는 조사와 어미 때문에 영어보다 까다롭다.,23,8,25
2,신조어와 외래어가 많은 SNS 텍스트는 전처리가 중요하다.,26,7,27
3,형태소 분석은 품사 정보를 함께 얻을 수 있어 텍스트 마이닝에 유리하다.,30,10,34
4,도메인에 따라 최적 토크나이저가 달라지므로 비교 실험이 필요하다.,29,10,35
5,챗지피티가 신조어를 잘 처리하는지 확인해 봅시다.,22,8,26


In [6]:

# --- 형태소 준비(Mecab→Okt 폴백) ---
from konlpy.tag import Okt
tokenizer_name = "okt"
okt = Okt()
mecab = None
try:
    from konlpy.tag import Mecab
    mecab = Mecab()
    tokenizer_name = "mecab"
except Exception as e:
    print("[알림] Mecab 사용 불가 → Okt로 진행합니다.")
print("사용 형태소 분석기:", tokenizer_name)

def morph_tokens(text, stem=True):
    if tokenizer_name == "mecab":
        return [t for t in mecab.morphs(text) if len(t)>=2]
    else:
        return [t for t in okt.morphs(text, stem=stem) if len(t)>=2]

# --- 말뭉치 준비 & 형태소로 1차 분절 ---
SENTS = [
    "오늘은 비가 오지만, 축구 경기는 예정대로 진행됐다.",
    "한글 토큰화는 조사와 어미 때문에 영어보다 까다롭다.",
    "신조어와 외래어가 많은 SNS 텍스트는 전처리가 중요하다.",
    "형태소 분석은 품사 정보를 함께 얻을 수 있어 텍스트 마이닝에 유리하다.",
    "도메인에 따라 최적 토크나이저가 달라지므로 비교 실험이 필요하다.",
]
morph_lines = [" ".join(morph_tokens(s)) for s in SENTS]
open("morph_corpus.txt","w",encoding="utf-8").write("\n".join(morph_lines))

# --- 형태소 토큰 시퀀스를 대상으로 SentencePiece Unigram 학습 ---
import sentencepiece as spm, pandas as pd
spm.SentencePieceTrainer.Train(
    input="morph_corpus.txt",
    model_prefix="sp_on_morph",
    model_type="unigram",
    vocab_size=76,  # Reduced vocab_size to 80
    character_coverage=0.9995
)
sp = spm.SentencePieceProcessor()
sp.load("sp_on_morph.model")

# --- 새 문장에 적용: (1) 형태소 → (2) SP compress ---
test = "챗지피티가 신조어를 잘 처리하는지 확인해 봅시다."
m1 = morph_tokens(test)
m2 = sp.EncodeAsPieces(" ".join(m1))
print("1단계(형태소):", m1)
print("2단계(형태소→SP):", m2, " | 길이:", len(m2))

# --- 효과 비교: 원문 → 형태소 → 형태소SP 길이 ---
import numpy as np, pandas as pd
rows=[]
for s in SENTS+[test]:
    t0 = list(s.replace(" ",""))             # 대충 문자 단위 길이(비교용)
    t1 = morph_tokens(s)
    t2 = sp.EncodeAsPieces(" ".join(t1))
    rows.append({"text": s, "len_char": len(t0), "len_morph": len(t1), "len_morph_sp": len(t2)})
pd.DataFrame(rows)

사용 형태소 분석기: mecab
1단계(형태소): ['지피', '신조어', '처리', '는지', '확인', '봅시다']
2단계(형태소→SP): ['▁', '지', '피', '▁', '신', '조', '어', '▁', '처', '리', '▁', '는', '지', '▁', '확', '인', '▁', '봅시', '다']  | 길이: 19


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: morph_corpus.txt
  input_format: 
  model_prefix: sp_on_morph
  model_type: UNIGRAM
  vocab_size: 76
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  

,text,len_char,len_morph,len_morph_sp
0,"오늘은 비가 오지만, 축구 경기는 예정대로 진행됐다.",23,7,21
1,한글 토큰화는 조사와 어미 때문에 영어보다 까다롭다.,23,8,24
2,신조어와 외래어가 많은 SNS 텍스트는 전처리가 중요하다.,26,6,23
3,형태소 분석은 품사 정보를 함께 얻을 수 있어 텍스트 마이닝에 유리하다.,30,8,27
4,도메인에 따라 최적 토크나이저가 달라지므로 비교 실험이 필요하다.,29,10,32
5,챗지피티가 신조어를 잘 처리하는지 확인해 봅시다.,22,6,19
